# Week 5: Machine Learning 2

There are innumerable machine learning models (and algorithms for fitting them to data) out there, and each one has something special about it that makes it suitable to a specific type of problem. To apply machine learning and get some initial results is fairly straight forward. Getting under the hood, however, requires a bit of work. This week we will look into how Decision Trees and PCA works. In the exercises today you will:

* Implement a standard decision tree mechanism
* Play around with a neural network

[**Feedback**](http://ulfaslak.com/vent)

## Exercises

**Get started**: Load the data you used last week to classify hero/villainness. It should consist of two arrays, one of dimensions ($N_{characters}$, $N_{teams}$) which is your feature matrix that has the team alliances of each character as a row vector of ones and zeros, and another of dimensions ($N_{characters}$, ) which is your target array that gives whether a character is a hero (1) or a villain (0). You can either load the data or copy/paste the code that generates it.

In [39]:
import re, os
import numpy as np

def get_alliances(char, faction=None):
    """Return list of alliances for Marvel character."""
    
    if faction is None:
        for faction in ["heroes", "ambiguous", "villains"]:
            faction_chars = [c[:-4] for c in os.listdir("./data/%s" % faction)]
            if char in faction_chars:
                break
    
    # Load character markup
    with open("./data/%s/%s.txt" % (faction, char)) as fp:
        markup = fp.read()

    # Get alliance field
    alliances_field = re.findall(r"alliances[\w\W]+?\|.+=", markup)
    if alliances_field == []:
        return []

    # Extract teams from alliance field
    return [t[2:-1] for t in re.findall(r"\[\[.+?[\]\|]", alliances_field[0][10:])]

In [7]:
all_teams=[]
seen = set(all_teams)

#add all unique teams to list
for faction in ["heroes", "ambiguous", "villains"]:
    faction_chars = [c[:-4] for c in os.listdir("./data/%s" % faction)]
    
    for char in faction_chars:
        alliances = get_alliances(char)
        
        for team in alliances:
            seen.add(team)

for team in seen:
    all_teams.append(team)

all_teams.sort()    
  
print(len(all_teams))    
#print(all_teams)   

622


In [40]:
def encode_alliance(char):
    """returns vector of 0,1 whether char is in team referenced to all_teams array"""
    vector = []
    alliances = get_alliances(char)
    c = 0
    
    for team in all_teams:
        if team not in alliances:
            vector.append(0)
        else:
            vector.append(1)
            c+=1
    
    
    return vector, c

In [49]:
#create matrix
matrix = []
target = []
chars = []
for faction in ["heroes", "villains"]:
    faction_chars = [c[:-4] for c in os.listdir("./data/%s" % faction)]
    
    for char in faction_chars:
        vector, length = encode_alliance(char)
   
        #only add characters in a team
        if length >= 1:       
            matrix.append(vector)
            chars.append(char)
        
            #append the faction of the character to target
            if faction == "heroes":
                target.append(1)
            else:
                target.append(0)

In [50]:
#load check
print(np.array(matrix))
print("\n")
print(np.array(target))

print(np.shape(matrix))
print(np.shape(target))

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


[1 1 1 ... 0 0 0]
(1096, 622)
(1096,)


### Part 1: Decision trees

In this exercise you will implement the decision making algorithm of a decision tree classifier, step by step.

>**Ex. 5.1.1**: Read about [Shannon entropy](https://en.wikipedia.org/wiki/Entropy_(information_theory)).
1. What is it? How is it defined mathematically (write out the formula in LateX formatting)?
> The entropy can be written as
> $$ \text{shanon entropy }H(X)=- \sum^n_{i=1}P(x_i)\log_b P(x_i)$$
> where $b$ is the base of the logarithm used (ie. 2, $e$, 10)
>
> input: probability vector (a list of values between 0 and 1 which sums to 1)
>
> output: entropy (the randomness of the result)
>
> this can be used in the construction of decision trees to score whether a question is good as it tells how it splits the data (low entropy is good)
>
> ex. p = [1,0]
> $$ \text{ Entropy } = -(1 * log_2(1) +0 * log_2(0)) = 0 $$
>


2. Write a function that computes the Shannon-entropy of a probability vector. Compute the Shannon entropy of `p=[0.4, 0.6]`.

In [51]:
import numpy as np

def shanon_entropy(p):
    entropy = 0
    
    for i in p:
        #if entry is 0 entropy must be 1
        if i == 0:
            return 1
        
        entropy += i * np.log2(i)
        
    return -entropy

In [52]:
p=[0.4, 0.6]
shanon_entropy(p)

0.9709505944546686

>**Ex. 5.1.2**: Split your data into two subsets. One where characters are affiliated with X-men and one where they are not.
1. What is the entropy of target labels in each subset?
2. What is the weighted average entropy of the split?
3. Write a function that computes the weighted average entropy of a split, given the data and team (name or id) on which to split the data.
4. Plot the distribution of split entropy for all features. Comment on the result. My figure looks [like this](http://ulfaslak.com/computational_analysis_of_big_data/exer_figures/example_6.2.2.4.png).

In [53]:
#split data on team
def split_team(team):
    #find team ind
    ind = 0
    for i in all_teams:
        if i == team:
            break
        ind+=1
    
    team = []
    team_ind = []
    
    non_team = []
    non_ind = []
    
    x_ind=0
    for char in matrix:
        if char[ind] == 1:
            team.append(char)
            team_ind.append(x_ind)
        else:
            non_team.append(char)
            non_ind.append(x_ind)
        x_ind+=1
    
    return team, team_ind, non_team, non_ind

In [54]:
#split data into X-men and non-X-Men
X_men, X_men_ind, reg_men, reg_men_ind = split_team('X-Men')

In [55]:
len(X_men_ind)

87

In [56]:
#grab target values
X_men_target=[]
reg_men_target=[]

for i in X_men_ind:
    X_men_target.append(target[i])

for i in reg_men_ind:
    reg_men_target.append(target[i])

In [57]:
len(X_men_target)

87

In [58]:
#find ratio of 0-1 for both sets
def hero_ratio(target):
    total = len(target)
  
    #team was emptyl entropy is 1
    if total == 0:
        return [1,0]
        
    ones = 0
    for i in target:
        if i == 1:
            ones+=1
    
    ones_ratio = ones/total
    zero_ratio = 1-ones_ratio
    return [zero_ratio, ones_ratio]

In [59]:
print("villian / hero ratio")

print("x-men ratios")
X_ratio = hero_ratio(X_men_target)
print(X_ratio)

print("other ratios")
reg_ratio = hero_ratio(reg_men_target) 
print(reg_ratio)

villian / hero ratio
x-men ratios
[0.08045977011494254, 0.9195402298850575]
other ratios
[0.5441030723488602, 0.45589692765113976]


In [60]:
#1
#entropy
X_entropy = shanon_entropy(X_ratio)
reg_entropy = shanon_entropy(reg_ratio)

print(X_entropy)
print(reg_entropy)

0.40379715049939247
0.9943803822497996


In [61]:
#2 
#weighted sum
def wsum_entropy(a, b, a_ent, b_ent):
    total = len(a) + len(b)
   
    if len(a) == 0:
        return 1
    
    w_a = len(a)/total
    w_b = len(b)/total
    
    return w_a * a_ent + w_b * b_ent

In [62]:
#NOTE: regular target includes characters with no team
wsum_entropy(X_men_target, reg_men_target, X_entropy, reg_entropy)

0.9475001439630428

In [63]:
#3
#weighted entropy of teams
def team_wsum_entropy(team):
    team, team_ind, non_team, non_ind = split_team(team)
   
    team_target=[]
    non_target=[]

    for i in team_ind:
        team_target.append(target[i])

    for i in non_ind:
        non_target.append(target[i])
        
    #ratios
    team_ratio = hero_ratio(team_target)
    non_ratio = hero_ratio(non_target)
    
    #entropy
    team_entropy = shanon_entropy(team_ratio)
    non_entropy = shanon_entropy(non_ratio)
    
    #weighted sum
    return wsum_entropy(team_target, non_target, 
                        team_entropy, non_entropy)

In [64]:
team_wsum_entropy('Avengers (comics)')

0.9353200324332954

>**Ex. 5.1.3**: Print the maximum entropy path of a decision tree.
>
>1. Implement the following pseudocode and print the output:<br><br>
>Step 1. Find `team` that gives lowest split entropy for `data`. Print `team`.<br>
>Step 2. Split `data` on `team`, to produce `data0` and `data1`. Print the entropy of each, as well as their weighted avg. entropy.<br>
>Step 3. Overwrite the `data` variable with either `data0` or `data1`, depending on which has the highest entropy.<br>
>Step 4. Stop if there are less than 5 datapoints in `data`. Otherwise start over from 1.<br><br>
>My output looks [like this](http://ulfaslak.com/computational_analysis_of_big_data/exer_figures/example_6.2.3.1.png) for the first five splits.<br><br>
>
>2. Comment on decision path your code takes: How splits are there? Do you notice anything interesting about the final splits? Why do we choose to stop splitting before `data` get smaller than 5?
>3. Train a `sklearn.tree.DecisionTreeClassifier` classifier on the dataset. Initiate the classifier with `criterion='entropy'`. What are the most important features of this classifier? How does this line up with the order of the order of splits you just printed (a comment is fine)?

In [70]:
lowest_ent = 1
lowest_team = ''
for team in all_teams:
    ent = team_wsum_entropy(team)
    if ent < lowest_ent:
        lowest_ent = ent
        lowest_team = team

print(lowest_ent)
print(lowest_team)

#split on lowest ent team
data0, data0_ind, data1_team, data1_ind = split_team(lowest_team)

0.9353200324332954
Avengers (comics)


### Part 2: Logistic regression

Logistic regressions are great baseline models for comparing how well a more complicated model works.
They are literally just linear regressions where the output is *squeezed* through a `sigmoid` function,
so the returned value is between 0 and 1 (which can be interpreted as a probability).

>**Ex. 5.2.1**: Implement a logistic regression model.
Below I have implemented a *linear* regression model which takes *two* input parameter.
Create another function, `logistic_regression` that takes again two input
variables and returns a value between 0 and 1. Demonstrate that it works by inputting
the data, `x=[1, 1]`, and parameters, `w0=1`, `w1=1` and `w2=0`, below, and show that it gives 0.88.
>
>*Hint*: The `sigmoid` function can look like this:
>
>        def sigmoid(x):
>            return 1 / (1 + np.exp(-x))

In [ ]:
def linear_regression(x, w0, w1, w2):
    return x[0] * w0 + x[1] * w1 + w2

# example
x = [1, 1]
w0 = 1; w1 = 1; w2 = 0
linear_regression(x, w0, w1, w2)

>**Ex. 5.2.2**: *Fit* a logistic regression! You can use the `scipy` module `scipy.optimize.curve_fit`
to fit a model to some data (i.e. find the best parameter values `w`). Below I have implemented an
example of how to fit some data to the linear regression above:


In [ ]:
from scipy.optimize import curve_fit
import numpy as np

def generate_X_linear(N=200):
    """A little function that creates some data."""
    x = np.vstack([
        np.random.normal([-2, -2], 1, size=(int(N/2), 2)),
        np.random.normal([2, 2], 1, size=(int(N/2), 2))
    ])

    y = np.array([0] * int(N/2) + [1] * int(N/2))
    
    return x, y

# Generate input and output data
x, y = generate_X_linear()

optimal_params, cov_params = curve_fit(linear_regression, x.T, y)
optimal_params

> Use the `logistic_regression` function you wrote in the previous exercise and fit it to `x` and `y`.
> Store the optimal parameters in a variable called `optimal_params` (like above), 
> then run the code cell below to plot the predictions. Mine looks like [this](https://ulfaslak.com/computational_analysis_of_big_data/exer_figures/example_5.5.2.png).

In [ ]:
import matplotlib.pylab as plt
plt.scatter(x[:, 0], x[:, 1], c=[logistic_regression(x_, *optimal_params) for x_ in x])
plt.show()

### Part 3 (extra): Neural networks

These days, neural networks are probably the hottest item in machine learning, and for good reason. Neural networks *can* be a bit of a mouthful to understand, and since I didn't talk about them in this course much (if at all), you can solve this problem if you have sufficient interest for it. That said, it is a skill well worth investing in.

We will be using the deep learning library **PyTorch** to build some neural networks with which we can play around with. *Nerdnote: Why not something more high-level such as Keras with a Tensorflow backend? Well PyTorch is easier to install. And using it, it's clearer what actually happens when you fit a neural network. Finally, for those taking the ANN course it's good to see something new.*

To get torch running you need to first install it. It's should be fairly straight forward, but depending on
your machine you may have to run different commands to install it. Check out the installation guide [here](https://pytorch.org/).

Once you've installed it you should be able to go:

In [71]:
import torch

ModuleNotFoundError: No module named 'torch'

No errors? Great! Then let's make a small neural network that we know all to well at this point, and see if we can classify points with it. First, we generate some data.

In [ ]:
def generate_X_nonlinear(N=200, R=5):

    X_inner = torch.randn(int(N/2), 2)

    X_outer = torch.tensor([
        [R*np.cos(theta), R*np.sin(theta)]
        for theta in np.linspace(0, 2 * np.pi, int(N/2))
    ]) + torch.randn(int(N/2), 2)

    X = torch.cat([X_inner, X_outer], dim=0)
   
    y = torch.cat([
        torch.zeros(int(N/2)).reshape(-1, 1),
        torch.ones(int(N/2)).reshape(-1, 1)
    ])
    
    return X, y

# Number of training datapoints
N = 500

# Generate the data (note that code is using torch arrays now)
x, y = generate_X_nonlinear(N)

Using this data, we can now set up a neural network and train it. We are not going to do anything fancy here, just make a simple 2-layer feed forward neural network with 2 input neurons, 3 hidden neurons and 1 output neuron.

In [ ]:
# The layers and their number of neurons
sizes = [2, 3, 1]

We then need to define the model. PyTorch has an API called `Sequential` that makes this pretty easy. Try to get an idea of what the below code is doing.

In [ ]:
model = torch.nn.Sequential(
    torch.nn.Linear(sizes[0], sizes[1]),
    torch.nn.Sigmoid(),
    torch.nn.Linear(sizes[1], sizes[2]),
    torch.nn.Sigmoid()
)

Finally we need to define a loss function. We are just going to use the sum of squared errors, and again PyTorch has got an implementation we can pick right off the shelf.

In [ ]:
loss_fn = torch.nn.MSELoss(reduction='sum')

And then we can just train it! We pick a learning rate parameter (which is how big the steps we take during
gradient descent are), define an *optimizer* which is a wrapper for training that abstracts away all the
usual steps. The `epochs` are simply the number of times we train on the entire dataset. Then we are ready to
train!

In [ ]:
# Hyper-parameters
learning_rate = 1e-1
epochs = 100
mini_batch_size = 100

# Optimization wrapper
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train
for t in range(epochs):
    
    # Randomly permute the row indices to get something like:
    # tensor([16214, 18491, 16308,  ..., 19629, 17565, 24696])
    permutation = torch.randperm(x.size()[0])
    
    # Start looping over the mini-batches! Each index `k` is
    # `mini_batch_size` values apart.
    for k in np.arange(0, x.size()[0], mini_batch_size):
        
        # Extract mini-batch data. The rest is the same
        mini_batch_indices = permutation[k:k+mini_batch_size]
        x_ = x[mini_batch_indices, :]
        y_ = y[mini_batch_indices, :]
        
        # Forward pass
        y_pred = model(x_)

        # Compute loss
        loss = loss_fn(y_pred, y_)

        # Before the backward pass, use the optimizer object to zero all of the
        # gradients for the variables it will update (which are the learnable
        # weights of the model). This is because by default, gradients are
        # accumulated in buffers (i.e, not overwritten) whenever .backward()
        # is called. Checkout docs of torch.autograd.backward for more details.
        optimizer.zero_grad()

        # Backward pass
        loss.backward()

        # Calling the step function on an Optimizer makes an update to its
        # parameters
        optimizer.step()
        
        
    # Print progress (here evaluating on all the data so we can compare)
    if t % 10 == 0:
        loss = loss_fn(model(x), y)
        print(t, "train:", loss.item())

Note that the errors we are showing here are the sum of squared errors. You'd have to jump through a small hoop (which is very doable) to get the accuracy. Also, this is the error on the training data. So there's no guarantee that we don't overfit here.

And now we can visualize the predictions next to the true labels!

In [ ]:
%matplotlib inline
from scipy.interpolate import interp1d
import matplotlib.pylab as plt

class cmap_in_range:
    """Create map to range of colors inside given domain.

    Example
    -------
    >>> cmap = cmap_in_range([0, 1])
    >>> cmap(0.1)
    (0.30392156862745101, 0.30315267411304353, 0.98816547208125938, 1.0)
    """
    def __init__(self, cmap_domain, cmap_range=[0, 1], cmap_style='rainbow'):
        self.cmap_domain = cmap_domain
        self.cmap_range = cmap_range
        self.m = interp1d(cmap_domain, cmap_range)
        self.cmap = plt.get_cmap(cmap_style)
        
    def __call__(self, value):
        if not self.cmap_domain[0] <= value <= self.cmap_domain[1]:
            raise Exception("Value must be inside cmap_domain.")
        return self.cmap(self.m(value))

In [ ]:
cmap = cmap_in_range([0, 1])

y_true = y.reshape(-1).numpy()
y_pred = model(x).data.numpy().reshape(-1)

plt.figure(figsize=(9, 3))

plt.subplot(1, 2, 1)
plt.title("True", fontsize=12)
plt.scatter(x[:, 0], x[:, 1], color=list(map(cmap, y_true)))

plt.subplot(1, 2, 2)
plt.title("Predicted", fontsize=12)
plt.scatter(x[:, 0], x[:, 1], color=list(map(cmap, y_pred)))
plt.show()

Cooool! Alright. Your turn...

> **Ex. 5.3.1**: Can you fit a neural network like the above (maybe with more or less layers and different number of hidden neurons in each layer) to the marvel data to predict good vs. evil?